In [54]:
import pandas as pd
from io import StringIO
from urllib.request import urlopen
import sqlite3

In [55]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', False)

In [56]:
def get_source(url):
    response = urlopen(url)
    html = response.read()
    return html.decode()

def get_tables_from_html(html):
    html_io = StringIO(html)
    return pd.read_html(html_io)

def save_tables_to_sqlite(tables, db_name):
    conn = sqlite3.connect(db_name)
    for name, table in tables.items():
        table.to_sql(name, conn, if_exists='replace', index=False)
    conn.close()

def load_tables_from_sqlite(db_name):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    dataframes = {}
    for table_name in tables:
        table_name = table_name[0]
        df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
        dataframes[table_name] = df

    conn.close()
    return dataframes

In [57]:
url = 'https://www.ibge.gov.br/indicadores#variacao-do-pib'
html_content = get_source(url)
tabelas = get_tables_from_html(html_content)

In [58]:
dict = {f'data_{i+1}': tab for i, tab in enumerate(tabelas)}

for tab_name, df in dict.items():
    dict[tab_name] = df.iloc[::2]
    dict[tab_name] = dict[tab_name].applymap(lambda x: x.replace('Último', '').replace('Anterior', '').replace('12 meses', '').replace('No ano', ''))

In [59]:
save_tables_to_sqlite(dict, 'tabelas_ibge.db')

In [60]:
dataframes = load_tables_from_sqlite('tabelas_ibge.db')

In [66]:
dataframes['data_1']

,Indicadores econômicos,Último,Anterior,12 meses,No ano
0,IPCA (%),"0,38 abr 2024","0,16 mar 2024","3,69","1,80"
1,INPC (%),"0,37 abr 2024","0,19 mar 2024","3,23","1,95"
2,IPCA-15 (%),"0,44 mai 2024","0,21 abr 2024","3,70","2,12"
3,IPP (%),"0,74 abr 2024","0,35 mar 2024","-3,08","0,99"
4,Custo do m² (%),"0,41 abr 2024","0,07 mar 2024","2,51","0,82"
5,Variação do PIB (%),"2,5 1º tri 2024","2,1 4º tri 2023","2,5","2,5"
6,PIB per capita (R$),"42.247,52 2021","35.935,69 2020",-,-
7,Indústria (%),"-0,5 abr 2024","0,9 mar 2024","1,5","3,5"
8,Comércio (%),"0,0 mar 2024","1,0 fev 2024","2,5","5,9"
9,Serviços (%),"0,4 mar 2024","-0,9 fev 2024","1,4","1,2"


In [67]:
dataframes['data_2']

,Indicadores sociais,Último,Anterior,12 meses,No ano
0,Desemprego (%),"7,9 1º tri 2024","7,4 4º tri 2023",-,-
1,Escolarização (%),"99,4 2023","99,4 2022",-,-
2,Analfabetismo (%),"5,4 2023","5,6 2022",-,-
3,Fecundidade (filhos por mulher),"1,76 2021","1,76 2020",-,-
4,Mortalidade infantil (óbitos a cada mil nascidos vivos),"11,20 2021","11,56 2020",-,-


In [68]:
dataframes['data_3']

,Indicadores agropecuários,Último,Anterior,12 meses,No ano
0,Produção de leite (litros),34.609.218 2022,35.183.066 2021,-,-
1,Produção de ovos (mil dúzias),4.886.564 2022,4.821.802 2021,-,-
2,Bovinos (cabeças),234.352.649 2022,224.601.992 2021,-,-
3,Suínos (cabeças),44.393.930 2022,42.550.892 2021,-,-
4,Galináceos (cabeças),1.586.047.875 2022,1.528.031.685 2021,-,-
5,Banana (Mil Reais),11.918.249 2022,9.993.648 2021,-,-
6,Café (Mil Reais),51.813.884 2022,34.829.901 2021,-,-
7,Cana-de-açúcar (Mil Reais),93.478.262 2022,75.270.110 2021,-,-
8,Laranja (Mil Reais),14.367.012 2022,12.537.708 2021,-,-
9,Milho (Mil Reais),137.743.946 2022,116.141.528 2021,-,-
